In [3]:
!pip install llama-index==0.9.48 
!pip install transformers
!pip install torch sentence-transformers
!pip install pyvis
import os
from llama_index.llms import OpenAI
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
import pandas as pd
pd.set_option("display.max_colwidth", None)

  Using cached numpy-1.22.4-cp39-cp39-macosx_10_15_x86_64.whl.metadata (2.0 kB)
Using cached numpy-1.22.4-cp39-cp39-macosx_10_15_x86_64.whl (17.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.22.4 which is incompatible.
trulens-eval 0.27.2 requires numpy>=1.23.5, but you have numpy 1.22.4 which is incompatible.


In [4]:
from openai import OpenAI
import httpx
os.environ["OPENAI_API_BASE"] = 'https://api.xty.app/v1'
os.environ["OPENAI_API_KEY"] = 'sk-x'

In [5]:
# Load the text
from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader(
    input_files=["peter_pan.txt"]
).load_data()

In [4]:
from llama_index.node_parser import SentenceWindowNodeParser

# Create the sentence window node parser
# Set the retrieval granularity
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [8]:
from llama_index.llms import OpenAI
llm = OpenAI(model="gpt-4", temperature=0.1)

In [1]:
# Prepare for embedding
!pip install -U langchain-openai
!pip install openai -y
!pip install langchain_experimental
!pip install "langchain[docarray]" 

   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/291.3 kB ? eta -:--:--
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/lilewill/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/Users/lilewill/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/Users/lilewill/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/Users/lilewill/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "/Users/lilewill/opt/anaconda3/lib/python3.9/http/client.py", line 463, in read
    n = self.readinto(b)
  File "/Users/lilewill/opt/anaconda3/lib/python3.9/http/client.py", line 507, in readinto
    n

In [2]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
# Set the sentence context
sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embeddings,
    node_parser=node_parser,
)

In [ ]:
# Set the vector index store of Qdrant or others
!pip install llama-index-vector-stores-qdrant
import qdrant_client
client = qdrant_client.QdrantClient(location=":memory:",
                                    search_type="mmr")

In [18]:
# Embedding
from llama_index import VectorStoreIndex
from llama_index import Document

vector_store = QdrantVectorStore(client=client, collection_name="peter_pan")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

document = Document(text="\n\n".join([doc.text for doc in documents]))
sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

In [19]:
# Prepare Postprocessor and Reranker 
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage
from llama_index.readers.web import TrafilaturaWebReader
from llama_index import Document
import numpy as np
import nest_asyncio
from llama_index.response_synthesizers import ResponseMode, get_response_synthesizer

# BAAI/bge-reranker-base
# link: https://huggingface.co/BAAI/bge-reranker-base
rerank = SentenceTransformerRerank(
    top_n=3, model="BAAI/bge-reranker-base"
)

/environment/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# Prepare the prompt template
from llama_index import PromptTemplate
from llama_index.response_synthesizers import TreeSummarize

qa_prompt_tmpl = (
"""Based on the context to answer a claim is TRUE or False, and give me your reasons. 
Make your answer concise. 
Claim: {query_str} 
Context: {context_str} 
Answer:
"""
)
prompt_w = PromptTemplate(qa_prompt_tmpl)

In [21]:
# Set the response synthesizer
from llama_index.response_synthesizers import ResponseMode, get_response_synthesizer
response_synthesizer = get_response_synthesizer(service_context=sentence_context,
                                                response_mode=ResponseMode.TREE_SUMMARIZE,
                                                summary_template=prompt_w,
                                                use_async=False,
                                                verbose=True)

In [142]:
# Create the query engine
sentence_window_engine = sentence_index.as_query_engine(
    search_type="mmr", 
    node_postprocessors=[postproc, rerank],
    response_synthesizer = response_synthesizer
)

In [36]:
# Test the Q&A
window_response = sentence_window_engine.query(
    "Is Peter Pan a brave person? "
)
from llama_index.response.notebook_utils import display_response

display_response(window_response)

/environment/miniconda3/lib/python3.10/abc.py:123: RuntimeWarning: coroutine 'run_async_tasks.<locals>._gather' was never awaited
  return _abc_subclasscheck(cls, subclass)
/environment/miniconda3/lib/python3.10/abc.py:123: RuntimeWarning: coroutine 'BaseQueryEngine.aquery' was never awaited
  return _abc_subclasscheck(cls, subclass)


1 text chunks after repacking


**`Final Response:`** TRUE, Peter Pan is a brave person.

Reasons:
1. Peter Pan is depicted as a leader who is respected and even revered by the Piccaninny warriors, indicating his courage and leadership in protecting his territory from pirates.
2. The text mentions Peter Pan saving Tiger Lily, showcasing his willingness to risk himself for others, a key trait of bravery.
3. The narrative sets Peter Pan against a "terrible man" (the pirate), highlighting a scenario where bravery is required to confront formidable enemies.

In [114]:
import pandas as pd
q = pd.read_csv('02-QC.csv')
q

,question,ground_truths
0,Wendy Darling met Peter Pan in the market for the first time.,"[""False. Wendy and Peter didn't meet in the market. Wendy Darling met Peter Pan when he flied in her family's window to retrieve his lost shadow for the fisrt time. ""]"
1,Wendy like Peter but Peter don't like her.,"[""False. Wendy like Peter indeed, Peter is also very fond of Wendy because of the stories she tells, which are mainly about Peter and his adventures. He is fascinated by Wendy's stories of him that he would often visit her to listen.""]"
2,Peter Pan has good parents.,"[""False. Peter's parents aren't mentioned. He is an orphan and his parents' names are never mentioned.""]"
3,Mr Darling changed after the chilren left. He had reason for doing so.,"[""True. He changes quite a lot after the children fly away: his guilt makes him re-examine his behavior and values, and he becomes more cheerful, easy-going, and sentimental, even “quixotic.” Missing his children brings him back to his own childhood.""]"
4,Peter Pan went to Neverland through flight in the story.,"[""True. Peter Pan gets to Neverland through flight, by travelling to the ""second star to the right, and straight on till morning."" Neverland is a small mythical island where Peter Pan and the Lost Boys live.""]"
5,Tinker Bell drunk the poison but she came back to life.,"[""True. Jane mourns the death of the tinker fairy in guilt and shame, and her belief in fairies begins to arise. Jane's newfound belief in fairies revives Tinker Bell.""]"


In [143]:
# Fact-checking
questions = q['question']
ground_truths = q['ground_truths']
contexts = []
answers = []
display = []

# Inference
for query in questions:
    window_response = sentence_window_engine.query(query)
    answers.append(display_response(window_response))


1 text chunks after repacking


**`Final Response:`** False.

The claim that Wendy Darling met Peter Pan in the market for the first time is not supported by the context provided or by the original story of Peter Pan. In J.M. Barrie's classic tale, Wendy Darling first encounters Peter Pan in her bedroom when he comes looking for his shadow. The passage provided does not mention a market at all; instead, it describes a scene where Peter is seen by Wendy before he flies away, and it also includes a conversation between Wendy and her mother about Peter Pan. Wendy's mother recalls Peter Pan from her own childhood, suggesting a mythical or legendary figure rather than someone Wendy would have met in a commonplace setting like a market.

1 text chunks after repacking


**`Final Response:`** TRUE

Reasons:
1. The context provided does not explicitly state Wendy's feelings towards Peter, but it is commonly known from the story of "Peter Pan" that Wendy harbors affection for Peter.
2. The context does show Peter pointing out drawbacks to Wendy, which could be interpreted as a lack of reciprocal affection or at least a lack of sensitivity towards Wendy's feelings. This suggests that Peter may not have the same affection for Wendy as she has for him.
3. Peter's focus in the conversation is more on the tactical situation with the pirates rather than on Wendy's feelings or reactions, further indicating that his priorities and affections may not align with Wendy's.

1 text chunks after repacking


...

**`Final Response:`** TRUE.

Reason: The context provided suggests a significant emotional impact on Mr. Darling due to the events surrounding the children and their safety, indicated by his actions and emotional state. Tying up Nana in the backyard and then sitting in the passage with his knuckles to his eyes shows a change in his behavior, likely driven by stress, worry, or guilt. This change in demeanor and actions after the children's involvement in the situation (as they were put to bed and the night-lights were lit) supports the claim that Mr. Darling changed after the children left, and he had reasons for doing so, rooted in the unfolding dramatic events and his emotional responses to them.

1 text chunks after repacking


**`Final Response:`** TRUE

Peter Pan and his companions indeed went to Neverland through flight. The context provided describes how the characters lost their ability to fly because they stopped believing, which implies that belief was a necessary component of their flight. Additionally, the passage mentions Wendy flying away with Peter, and the chapter titled "THE FLIGHT" suggests that flying is the method by which they travel to Neverland. The quote "Second to the right, and straight on till morning" is a famous line from the story that Peter Pan gives as the direction to Neverland, further supporting the idea that they flew there.

1 text chunks after repacking


**`Final Response:`** TRUE. Tinker Bell did drink the poison, but she came back to life. In the story of Peter Pan by J.M. Barrie, Tinker Bell drinks the poison that was intended for Peter Pan in order to save him. After she drinks it, she indeed begins to die. However, she is revived when Peter Pan calls upon children who believe in fairies to clap their hands, thus restoring her life through their belief and the power of their clapping. This act of collective belief and the magical nature of the story allow Tinker Bell to come back to life despite the lethal effect of the poison.

In [116]:
!pip uinstall ragas
!pip install ragas==0.0.22

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [144]:
from ragas.llama_index import evaluate # Please attention: this evaluate funtion is just for ragas==0.0.22

metrics = [faithfulness]

In [145]:
# Avoid the runtime error
import asyncio
import time
nest_asyncio.apply()

In [146]:
result = evaluate(sentence_window_engine, metrics, questions,answers)

1 text chunks after repacking
1 text chunks after repacking
1 text chunks after repacking
1 text chunks after repacking
1 text chunks after repacking
1 text chunks after repacking


Evaluating: 100%|██████████| 6/6 [00:26<00:00,  4.41s/it]


In [147]:
data = result.to_pandas()
data1 = data.to_dict()

In [148]:
questions = data['question']
ground_truths = q['ground_truths']
contexts = data['contexts']
answers = data['answer']
from datasets import Dataset 
from ragas import evaluate # This is the function of latest ragas
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness
)

metrics = [
    faithfulness,
    context_precision,
    context_recall,
    answer_relevancy,
    answer_correctness
]

In [149]:
data = {
    "question": questions,
    "answer": answers,
    'contexts': contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=metrics,
)

res_w1 = result.to_pandas()

Evaluating: 100%|██████████| 30/30 [00:37<00:00,  1.25s/it]


In [150]:
res_w1

,question,answer,contexts,ground_truth,faithfulness,context_precision,context_recall,answer_relevancy,answer_correctness
0,Wendy Darling met Peter Pan in the market for the first time.,"False.\n\nWendy Darling did not meet Peter Pan in the market for the first time. According to J.M. Barrie's novel ""Peter Pan,"" Wendy first encountered Peter Pan when he visited the nursery of the Darling family's home at night. Peter was there looking for his shadow which Wendy's mother, Mrs. Darling, had previously found and stored away. Wendy helps Peter by sewing his shadow back on, marking the beginning of their acquaintance and subsequent adventures. The text provided does not mention a market meeting, and the traditional story aligns with their meeting occurring in Wendy's home.","[“Then follow the leader,” he cried gaily. “Mind you, I am not sure that\nwe have a drawing-room, but we pretend we have, and it’s all the same.\n Hoop la!”\n\nHe went off dancing through the house, and they all cried “Hoop la!”\nand danced after him, searching for the drawing-room; and I forget\nwhether they found it, but at any rate they found corners, and they all\nfitted in.\n\n As for Peter, he saw Wendy once again before he flew away. He did not\nexactly come to the window, but he brushed against it in passing so\nthat she could open it if she liked and call to him. That is what she\ndid.\n\n, “Yes, he is rather cocky,” Wendy admitted with regret. Her mother had\nbeen questioning her.\n\n “But who is he, my pet?”\n\n“He is Peter Pan, you know, mother.”\n\nAt first Mrs. Darling did not know, but after thinking back into her\nchildhood she just remembered a Peter Pan who was said to live with the\nfairies. There were odd stories about him, as that when children died\nhe went part of the way with them, so that they should not be\nfrightened. She had believed in him at the time, but now that she was\nmarried and full of sense she quite doubted whether there was any such\nperson.\n\n]","[""False. Wendy and Peter didn't meet in the market. Wendy Darling met Peter Pan when he flied in her family's window to retrieve his lost shadow for the fisrt time. ""]",NaN,1.0,0.00,0.000000,0.236439
1,Wendy like Peter but Peter don't like her.,"Based on the context provided, the claim that Wendy likes Peter but Peter doesn't like her cannot be definitively proven true or false solely from the text given. The excerpt focuses more on the dynamics of their adventure and the concern for safety rather than explicitly detailing the personal feelings between Wendy and Peter. However, there are subtle indications of care and interaction, such as Wendy's concern and the group dynamics, but these do not directly address their personal feelings towards each other in a romantic sense. Therefore, without additional context specifically addressing Peter's feelings towards Wendy, I cannot accurately determine the truthfulness of the claim.","[Look at her in her chair, where she has\nfallen asleep. The corner of her mouth, where one looks first, is\nalmost withered up. Her hand moves restlessly on her breast as if she\nhad a pain there. Some like Peter best, and some like Wendy best, but I\nlike her best. Suppose, to make her happy, we whisper to her in her\nsleep that the brats are coming back. They are really within two miles\nof the window now, and flying strong, but all we need whisper is that\nthey are on the way. , “It is this, if we meet Hook in open fight, you must leave him to me.”\n\n“I promise,” John said loyally.\n\n For the moment they were feeling less eerie, because Tink was flying\nwith them, and in her light they could distinguish each other.\n Unfortunately she could not fly so slowly as they, and so she had to go\nround and round them in a circle in which they moved as in a halo.\n Wendy quite liked it, until Peter pointed out the drawbacks.\n\n “She tells me,” he said, “that the pirates sighted us before the\ndarkness came, and got Long Tom out.”\n\n“The big gun?”\n\n“Y

In [151]:
res_w1.describe()

,faithfulness,context_precision,context_recall,answer_relevancy,answer_correctness
count,3.0,6.000000,6.000000,6.000000,6.000000
mean,1.0,0.916667,0.291667,0.465652,0.461952
std,0.0,0.204124,0.458712,0.510273,0.282992
min,1.0,0.500000,0.000000,0.000000,0.215281
25%,1.0,1.000000,0.000000,0.000000,0.225232
50%,1.0,1.000000,0.000000,0.453957,0.413410
75%,1.0,1.000000,0.562500,0.929475,0.602204
max,1.0,1.000000,1.000000,0.949334,0.901970


## Knowledge Graph for RAG

In [6]:
from llama_index.graph_stores import SimpleGraphStore, Neo4jGraphStore, NebulaGraphStore
from llama_index import load_index_from_storage
from llama_index.readers.web import TrafilaturaWebReader
from llama_index import KnowledgeGraphIndex

In [13]:
from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader(
    input_files=["peter_pan.txt"]
).load_data()

In [14]:
service_context_g = ServiceContext.from_defaults(llm=llm, chunk_size=256)
graph_store = SimpleGraphStore() # Neo4jGraphStore, NebulaGraphStore
storage_context_g = StorageContext.from_defaults(graph_store=graph_store)

In [15]:
# Create Index
index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=3,
    include_embeddings=False
)

In [16]:
# Create graph
from pyvis.network import Network

g = index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)
net.show("no_embedding_KG.html")

no_embedding_KG.html


In [17]:
# Query with embeddings
e_index = KnowledgeGraphIndex.from_documents(
    documents,
    max_triplets_per_chunk=3,
    include_embeddings=True,
)

In [18]:
# Create graph
from pyvis.network import Network

ge = e_index.get_networkx_graph()
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(ge)
net.show("embedding_KG.html")

embedding_KG.html


In [159]:
# Create a KG keyword query engine
query_engine = index.as_query_engine(
    # setting to false uses the raw triplets instead of adding the text from the corresponding nodes
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
    response_synthesizer = response_synthesizer
)

In [160]:
# Prepare the queries
import pandas as pd
q = pd.read_csv('02-QC.csv')
q

,question,ground_truths
0,Wendy Darling met Peter Pan in the market for the first time.,"[""False. Wendy and Peter didn't meet in the market. Wendy Darling met Peter Pan when he flied in her family's window to retrieve his lost shadow for the fisrt time. ""]"
1,Wendy like Peter but Peter don't like her.,"[""False. Wendy like Peter indeed, Peter is also very fond of Wendy because of the stories she tells, which are mainly about Peter and his adventures. He is fascinated by Wendy's stories of him that he would often visit her to listen.""]"
2,Peter Pan has good parents.,"[""False. Peter's parents aren't mentioned. He is an orphan and his parents' names are never mentioned.""]"
3,Mr Darling changed after the chilren left. He had reason for doing so.,"[""True. He changes quite a lot after the children fly away: his guilt makes him re-examine his behavior and values, and he becomes more cheerful, easy-going, and sentimental, even “quixotic.” Missing his children brings him back to his own childhood.""]"
4,Peter Pan went to Neverland through flight in the story.,"[""True. Peter Pan gets to Neverland through flight, by travelling to the ""second star to the right, and straight on till morning."" Neverland is a small mythical island where Peter Pan and the Lost Boys live.""]"
5,Tinker Bell drunk the poison but she came back to life.,"[""True. Jane mourns the death of the tinker fairy in guilt and shame, and her belief in fairies begins to arise. Jane's newfound belief in fairies revives Tinker Bell.""]"


In [161]:
from llama_index import PromptTemplate
from llama_index.response_synthesizers import TreeSummarize

# Build the prompt

qa_prompt_tmpl = (
"""Based on the context to answer a claim is TRUE or False, and give me your reasons. 
If you don't know, just said you don't know.  
Claim: {query_str} 
Context: {context_str} 
Answer:
"""
)
prompt_kg = PromptTemplate(qa_prompt_tmpl)

In [162]:
# Cerate the response synthesizer
from llama_index.response_synthesizers import ResponseMode, get_response_synthesizer
response_synthesizer = get_response_synthesizer(service_context=service_context_g,
                                                response_mode=ResponseMode.TREE_SUMMARIZE,
                                                summary_template=prompt_kg,
                                                use_async=True,
                                                verbose=True)

In [163]:
# Generate answers
questions = q['question']
ground_truths = q['ground_truths']
answers = []
display = []

for query in questions:
    response = query_engine.query(query)
    answers.append(str(response))
    display.append(display_response(response))


1 text chunks after repacking


**`Final Response:`** FALSE

Reason: The provided knowledge graph does not contain any information or sequence that explicitly states that Wendy Darling met Peter Pan in the market for the first time. The sequences provided focus on various actions and attributes related to Peter Pan, Wendy, and other characters, but none mention a meeting in the market. The closest related information is about Wendy seeing Peter's shadow and interactions involving Mrs. Darling and Peter Pan, but nothing about a first meeting in a market setting.

1 text chunks after repacking


**`Final Response:`** TRUE

Reasons:
1. The claim states that Wendy likes Peter. This is supported by the context where Wendy is shown to have emotional responses related to Peter, such as when she "Cried, 'O Peter'" and "Felt, 'Sorry'" which indicates a caring or affectionate attitude towards Peter.

2. The claim also states that Peter does not like Wendy. While the provided context does not explicitly state Peter's feelings towards Wendy, the absence of direct interactions or positive predicates from Peter directed towards Wendy in the context suggests a lack of reciprocal affection or interest. Peter's actions and interactions are mostly centered around other characters like Tinker Bell and his own adventures, indicating his focus is elsewhere.

Thus, based on the emotional investment shown by Wendy towards Peter and the lack of reciprocal actions or expressions from Peter towards Wendy in the provided context, the claim can be considered true.

1 text chunks after repacking


...

**`Final Response:`** TRUE

Reasons:
1. The claim states that Mr. Darling changed after the children left, and he had reason for doing so. Although the provided knowledge sequence does not explicitly mention Mr. Darling's change or the reasons for it, we can infer a possible emotional impact based on the context of children leaving, which is a significant event in a parent's life.
2. The knowledge sequence mentions Peter Pan in relation to home and familial roles ("Came back for," "Mother"), suggesting a narrative involving family dynamics and possibly the absence of children from home. This context supports the idea that the departure of children (presumably including the Darling children involved with Peter Pan) could lead to significant emotional and behavioral changes in a parent like Mr. Darling.
3. The absence of direct information about Mr. Darling's change or reasons in the knowledge sequence does not disprove the claim but rather leaves room for reasonable inference based on typical human emotional responses to such family events.

Therefore, based on inference and typical emotional responses to children leaving home, the claim is considered TRUE.

1 text chunks after repacking


**`Final Response:`** TRUE.

The claim that Peter Pan went to Neverland through flight in the story is true. While the provided knowledge sequence does not explicitly state that Peter Pan flew to Neverland, it is a well-known part of the Peter Pan story by J.M. Barrie. In the original tale, Peter Pan flies to Neverland, and he teaches Wendy and her brothers to fly using pixie dust provided by Tinker Bell. The knowledge sequence mentions characters and elements like Tinker Bell and Peter's shadow that are associated with the flight to Neverland, supporting the context of the claim.

1 text chunks after repacking


**`Final Response:`** TRUE. In the story of Peter Pan, Tinker Bell drinks poison that was meant for Peter Pan. She does this to save him. After drinking the poison, Tinker Bell begins to die, but she is revived when children around the world are asked to clap their hands if they believe in fairies. The collective belief and clapping bring her back to life.

In [164]:
from ragas import evaluate
metrics = [answer_correctness]
data = {
    "question": questions,
    "answer": answers,
    'contexts': contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=metrics,
)

res_kg1 = result.to_pandas()

Evaluating: 100%|██████████| 6/6 [00:11<00:00,  1.84s/it]


In [165]:
res_kg1

,question,answer,contexts,ground_truth,answer_correctness
0,Wendy Darling met Peter Pan in the market for the first time.,"FALSE\n\nReason: The provided knowledge graph does not contain any information or sequence that explicitly states that Wendy Darling met Peter Pan in the market for the first time. The sequences provided focus on various actions and attributes related to Peter Pan, Wendy, and other characters, but none mention a meeting in the market. The closest related information is about Wendy seeing Peter's shadow and interactions involving Mrs. Darling and Peter Pan, but nothing about a first meeting in a market setting.","[“Then follow the leader,” he cried gaily. “Mind you, I am not sure that\nwe have a drawing-room, but we pretend we have, and it’s all the same.\n Hoop la!”\n\nHe went off dancing through the house, and they all cried “Hoop la!”\nand danced after him, searching for the drawing-room; and I forget\nwhether they found it, but at any rate they found corners, and they all\nfitted in.\n\n As for Peter, he saw Wendy once again before he flew away. He did not\nexactly come to the window, but he brushed against it in passing so\nthat she could open it if she liked and call to him. That is what she\ndid.\n\n, “Yes, he is rather cocky,” Wendy admitted with regret. Her mother had\nbeen questioning her.\n\n “But who is he, my pet?”\n\n“He is Peter Pan, you know, mother.”\n\nAt first Mrs. Darling did not know, but after thinking back into her\nchildhood she just remembered a Peter Pan who was said to live with the\nfairies. There were odd stories about him, as that when children died\nhe went part of the way with them, so that they should not be\nfrightened. She had believed in him at the time, but now that she was\nmarried and full of sense she quite doubted whether there was any such\nperson.\n\n]","[""False. Wendy and Peter didn't meet in the market. Wendy Darling met Peter Pan when he flied in her family's window to retrieve his lost shadow for the fisrt time. ""]",0.227223
1,Wendy like Peter but Peter don't like her.,"TRUE\n\nReasons:\n1. The claim states that Wendy likes Peter. This is supported by the context where Wendy is shown to have emotional responses related to Peter, such as when she ""Cried, 'O Peter'"" and ""Felt, 'Sorry'"" which indicates a caring or affectionate attitude towards Peter.\n\n2. The claim also states that Peter does not like Wendy. While the provided context does not explicitly state Peter's feelings towards Wendy, the absence of direct interactions or positive predicates from Peter directed towards Wendy in the context suggests a lack of reciprocal affection or interest. Peter's actions and interactions are mostly centered around other characters like Tinker Bell and his own adventures, indicating his focus is elsewhere.\n\nThus, based on the emotional investment shown by Wendy towards Peter and the lack of reciprocal actions or expressions from Peter towards Wendy in the provided context, the claim can be considered true.","[Look at her in her chair, where she has\nfallen asleep. The corner of her mouth, where one looks first, is\nalmost withered up. Her hand moves restlessly on her breast as if she\nhad a pain there. Some like Peter best, and some like Wendy best, but I\nlike her best. Suppose, to make her happy, we whisper to her in her\nsleep that the brats are coming back. They are really within two miles\nof the window now, and flying strong, but all we need whisper is that\nthey are on the way. , “It is this, if we meet Hook in open fight, you must leave him to me.”\n\n“I promise,” John said loyally.\n\n For the moment they were feeling less eerie, because Tink was flying\nwith them, and in her light they could distinguish each other.\n Unfortunately she could not fly so slowly as they, and so she had to go\nround and round them in a circle in which they moved as in a halo.\n Wendy quite liked it, until Peter pointed out the drawbacks.\n\n “She tells me,” he said, “that th

In [166]:
res_kg1.describe()

,answer_correctness
count,6.000000
mean,0.508566
std,0.333436
min,0.216524
25%,0.224947
50%,0.415035
75%,0.764761
max,0.961879


## Hybrid Search for KG RAG

In [200]:
# Create a KG (keyword + vector) hybrid query engine
query_engine_e = e_index.as_query_engine(
    include_text=True,
    retriever_mode="hybrid",
    response_mode="tree_summarize",
    similarity_top_k=5,
    explore_global_knowledge=True,
    response_synthesizer = response_synthesizer
)

In [201]:
# Generate answers using 
questions = q['question']
ground_truths = q['ground_truths']
answers = []
display = []

for query in questions:
    response = query_engine_e.query(query)
    answers.append(str(response))
    display.append(display_response(response))


2 text chunks after repacking
1 text chunks after repacking


**`Final Response:`** False.

Wendy Darling did not meet Peter Pan in the market for the first time. According to J.M. Barrie's novel "Peter Pan," Wendy first encountered Peter Pan in the nursery of her family's home. Peter Pan visited the nursery in search of his lost shadow, which is where he met Wendy and her brothers. There is no mention of a market in their initial meeting in the story.

2 text chunks after repacking
1 text chunks after repacking


**`Final Response:`** FALSE

Reasons:
1. The context suggests that Peter does have feelings for Wendy, as indicated by his jealousy when she talks about other boys, which is a sign of emotional attachment.
2. Peter's upset reaction to Tinker Bell harming Wendy out of jealousy further implies that he cares for Wendy's well-being, which is not consistent with the claim that he doesn't like her.
3. Although Peter refers to Wendy as his mother and himself as her son, this dynamic can be complex and does not necessarily negate the possibility of affectionate feelings; it may indicate his confusion about the nature of his feelings.
4. The complexity of Peter's feelings, including jealousy and protectiveness, suggests that he does have some form of attachment to Wendy, contradicting the claim that he does not like her.

2 text chunks after repacking
1 text chunks after repacking


...

**`Final Response:`** TRUE

Mr. Darling did indeed change after the children left. The context provided shows that Mr. Darling was deeply affected by the absence of his children. He felt remorse and guilt for his actions, particularly for his role in the events leading up to the children's departure. His decision to live in the dog's kennel as a form of self-imposed punishment demonstrates a significant change in his character. He went from being a proud and somewhat pompous figure to a humbled and remorseful one, showing deference to Nana the dog and seeking to atone for his behavior. This change was driven by his love for his children and his realization of the consequences of his actions.

2 text chunks after repacking
1 text chunks after repacking


**`Final Response:`** TRUE

The claim that Peter Pan went to Neverland through flight in the story is true. As mentioned, this is a fundamental aspect of J.M. Barrie's "Peter Pan." Peter Pan, along with Wendy, John, and Michael Darling, flies to Neverland after being sprinkled with pixie dust by Tinker Bell. The instruction to head towards the "second star to the right and straight on till morning" is iconic and directly relates to their flight to Neverland. This method of travel to Neverland is a well-known and integral part of the narrative.

1 text chunks after repacking


**`Final Response:`** TRUE

In the story of "Peter Pan," Tinker Bell does indeed drink poison that was meant for Peter Pan, and she nearly dies from it. However, she is revived when Peter asks children who believe in fairies to clap their hands to show their belief, which restores her strength and brings her back to life. This event is a pivotal moment that showcases the themes of belief and the power of collective faith in the narrative.

In [202]:
metrics = [answer_correctness]
data = {
    "question": questions,
    "answer": answers,
    'contexts': contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset, 
    metrics=metrics,
)

res_kg2 = result.to_pandas()

Evaluating: 100%|██████████| 6/6 [00:09<00:00,  1.64s/it]


In [203]:
res_kg2

,question,answer,contexts,ground_truth,answer_correctness
0,Wendy Darling met Peter Pan in the market for the first time.,"False.\n\nWendy Darling did not meet Peter Pan in the market for the first time. According to J.M. Barrie's novel ""Peter Pan,"" Wendy first encountered Peter Pan in the nursery of her family's home. Peter Pan visited the nursery in search of his lost shadow, which is where he met Wendy and her brothers. There is no mention of a market in their initial meeting in the story.","[“Then follow the leader,” he cried gaily. “Mind you, I am not sure that\nwe have a drawing-room, but we pretend we have, and it’s all the same.\n Hoop la!”\n\nHe went off dancing through the house, and they all cried “Hoop la!”\nand danced after him, searching for the drawing-room; and I forget\nwhether they found it, but at any rate they found corners, and they all\nfitted in.\n\n As for Peter, he saw Wendy once again before he flew away. He did not\nexactly come to the window, but he brushed against it in passing so\nthat she could open it if she liked and call to him. That is what she\ndid.\n\n, “Yes, he is rather cocky,” Wendy admitted with regret. Her mother had\nbeen questioning her.\n\n “But who is he, my pet?”\n\n“He is Peter Pan, you know, mother.”\n\nAt first Mrs. Darling did not know, but after thinking back into her\nchildhood she just remembered a Peter Pan who was said to live with the\nfairies. There were odd stories about him, as that when children died\nhe went part of the way with them, so that they should not be\nfrightened. She had believed in him at the time, but now that she was\nmarried and full of sense she quite doubted whether there was any such\nperson.\n\n]","[""False. Wendy and Peter didn't meet in the market. Wendy Darling met Peter Pan when he flied in her family's window to retrieve his lost shadow for the fisrt time. ""]",0.236154
1,Wendy like Peter but Peter don't like her.,"FALSE\n\nReasons:\n1. The context suggests that Peter does have feelings for Wendy, as indicated by his jealousy when she talks about other boys, which is a sign of emotional attachment.\n2. Peter's upset reaction to Tinker Bell harming Wendy out of jealousy further implies that he cares for Wendy's well-being, which is not consistent with the claim that he doesn't like her.\n3. Although Peter refers to Wendy as his mother and himself as her son, this dynamic can be complex and does not necessarily negate the possibility of affectionate feelings; it may indicate his confusion about the nature of his feelings.\n4. The complexity of Peter's feelings, including jealousy and protectiveness, suggests that he does have some form of attachment to Wendy, contradicting the claim that he does not like her.","[Look at her in her chair, where she has\nfallen asleep. The corner of her mouth, where one looks first, is\nalmost withered up. Her hand moves restlessly on her breast as if she\nhad a pain there. Some like Peter best, and some like Wendy best, but I\nlike her best. Suppose, to make her happy, we whisper to her in her\nsleep that the brats are coming back. They are really within two miles\nof the window now, and flying strong, but all we need whisper is that\nthey are on the way. , “It is this, if we meet Hook in open fight, you must leave him to me.”\n\n“I promise,” John said loyally.\n\n For the moment they were feeling less eerie, because Tink was flying\nwith them, and in her light they could distinguish each other.\n Unfortunately she could not fly so slowly as they, and so she had to go\nround and round them in a circle in which they moved as in a halo.\n Wendy quite liked it, until Peter pointed out the drawbacks.\n\n “She tells me,” he said, “that the pirates sighted us before the\ndarkness came, and got Long Tom out.”\n\n“The big gun?”\n\n“Yes. And of course they must see her light, and if they guess we are\nnear it they are sure to let fly.”\n\n“Wendy!”\n\n“John!”\n\n“Michael!”\n\n“Tell her to go away at once, Peter,” the th

In [205]:
res_kg2.describe()

,answer_correctness
count,6.000000
mean,0.579335
std,0.390419
min,0.217403
25%,0.226871
50%,0.540104
75%,0.938714
max,0.984352
